In [1]:
!pip install pyvi


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from pyvi import ViTokenizer
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import joblib
import optuna
import time
import os

In [21]:
os.makedirs("./Model", exist_ok=True)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.colheader_justify', 'left')

In [23]:
data = pd.read_csv("Dataset/vsa_food_rv_train_clean.csv", encoding="utf-8")
X = data["Comment"]
y = data["Rating"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2020, test_size=0.2)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10326,), (10326,), (2582,), (2582,))

In [24]:
for i in range(5):
    print(i, X_train.iloc[i])

0 chè ngon miễn bàn quán này có cái tủ chè xinh xinh mình rất thích sẽ quay lại vì thấy menu có nhiều món mới thử được 3 món
1 chắc tại đang đói nên mở ra thấy mùi gà thơm lừng nhìn ngon mlemm hết nấc chân gà thì ở đây ăn cũng giống những chỗ khác thôi không có gì đặc biệt à ngoài chân gà ra thì xem menu còn có rất nhiều món từ gà nữa nhé nói chung là khá ngon các bạn ở quanh đây nên order để ăn thử nhé
2 mình cũng từng đến đây ăn vài lần và có nhận xét như sau 1 đồ ăn khá hợp miệng mình tuy nhiên cơm hơi nát và cho ít nước sốt thì sẽ rất khó ăn 2 thái độ nhân viên từ chạy bàn cho đến order ở quầy đều không được thân thiện cho lắm 3 cái này cảm thấy khó chịu nhất này đó là nhân viên trông xe rất lười dắt xe cho khách mà có dắt thì dựng xe khách va đập vào nhau rồi với thái độ làm kiểu bực tức mong chủ quán chỉnh đốn lại nhân viên
3 20 ngàn-35
4 không gian quán thoáng mát sạch sẽ bún bò huế thực sự ngon và đậm vị xứ huế sẽ ghé thăm quán thêm nhiều lần nữa


In [25]:
class SVM_TFIDF:
    def __init__(self):
        self.vectorizer = None
        self.classifier = None

    def hyperp_tunning(self, X, y, n_trials):
        def objective(trial):
            # Khai báo siêu tham số cần tối ưu
            ngram_start = 1
            ngram_end = trial.suggest_int("tfidf__ngram_end", 1, 2)
            ngram_range = (ngram_start, ngram_end)
            min_df = trial.suggest_categorical("tfidf__min_df", [3, 5])
            max_df = trial.suggest_categorical("tfidf__max_df", [0.9, 0.95])
            sublinear_tf = trial.suggest_categorical("tfidf__sublinear_tf", [False, True])

            kernel = trial.suggest_categorical("svc__kernel", ["linear", "rbf"])
            C = trial.suggest_float("svc__C", 0.1, 5.0, log=True)
            gamma = trial.suggest_categorical("svc__gamma", ["scale", "auto"])

            pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(
                    tokenizer=ViTokenizer.tokenize,
                    ngram_range=ngram_range,
                    min_df=min_df,
                    max_df=max_df,
                    sublinear_tf=sublinear_tf
                )),
                ('svc', SVC(
                    class_weight='balanced',
                    kernel=kernel,
                    C=C,
                    gamma=gamma
                ))
            ])

            scores = cross_val_score(pipeline, X, y, cv=3, scoring="f1_macro")
            return scores.mean()

        print("*** Hyper-parameters tuning with Optuna ***")
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials)

        print("Best params:", study.best_params)
        return study.best_params
        
    def fit(self, X, y, n_trials=10):
        # Huấn luyện lại với tham số tốt nhất
        best_params = self.hyperp_tunning(X, y, n_trials)
        print("*** Training with the best params ***")
        start = time.time()
        self.vectorizer = TfidfVectorizer(
            tokenizer=ViTokenizer.tokenize,
            ngram_range= (1, best_params["tfidf__ngram_end"]),
            min_df=best_params["tfidf__min_df"],
            max_df=best_params["tfidf__max_df"],
            sublinear_tf=best_params["tfidf__sublinear_tf"]
        )
        X_vectorized = self.vectorizer.fit_transform(X)
        self.classifier = SVC(
            class_weight='balanced',
            kernel=best_params["svc__kernel"],
            C=best_params["svc__C"],
            gamma=best_params["svc__gamma"]
        )
        self.classifier.fit(X_vectorized, y)
        end = time.time()
        print(f"Training done after {(end - start):.2f}s")

    def predict(self, X):
        X_vectorized = self.vectorizer.transform(X)
        return self.classifier.predict(X_vectorized)

    def evaluate(self, y_true, y_pred):
        print("Evaluate for SVM classifier with TF-IDF vectorizer: ")
        print(classification_report(y_true, y_pred))


In [26]:
import warnings
warnings.filterwarnings("ignore", message=".*token_pattern.*")

In [27]:
svm = SVM_TFIDF()

print("========== RUNNING SVM - TFIDF ==========")
svm.fit(X_train, y_train)

file_path = "Model/SVM_TFIDF.pkl"
joblib.dump(svm, file_path)
size = os.path.getsize(file_path)/(1024 * 1024)
print(f"Model size : {size:.2f}MB")

[I 2025-12-14 17:46:26,572] A new study created in memory with name: no-name-e352c192-3a99-43be-b8df-937b0d0eb437


========== RUNNING SVM - TFIDF ==========
*** Hyper-parameters tuning with Optuna ***


[I 2025-12-14 17:47:52,918] Trial 0 finished with value: 0.7108444057569078 and parameters: {'tfidf__ngram_end': 1, 'tfidf__min_df': 3, 'tfidf__max_df': 0.95, 'tfidf__sublinear_tf': True, 'svc__kernel': 'linear', 'svc__C': 1.4292670704510377, 'svc__gamma': 'scale'}. Best is trial 0 with value: 0.7108444057569078.
[I 2025-12-14 17:51:00,380] Trial 1 finished with value: 0.8523167122767674 and parameters: {'tfidf__ngram_end': 2, 'tfidf__min_df': 5, 'tfidf__max_df': 0.9, 'tfidf__sublinear_tf': True, 'svc__kernel': 'linear', 'svc__C': 0.23950619283673805, 'svc__gamma': 'auto'}. Best is trial 1 with value: 0.8523167122767674.
[I 2025-12-14 17:52:21,739] Trial 2 finished with value: 0.7104897310717254 and parameters: {'tfidf__ngram_end': 1, 'tfidf__min_df': 5, 'tfidf__max_df': 0.95, 'tfidf__sublinear_tf': False, 'svc__kernel': 'linear', 'svc__C': 1.4446904941010474, 'svc__gamma': 'auto'}. Best is trial 1 with value: 0.8523167122767674.
[I 2025-12-14 17:53:41,261] Trial 3 finished with value:

Best params: {'tfidf__ngram_end': 2, 'tfidf__min_df': 3, 'tfidf__max_df': 0.9, 'tfidf__sublinear_tf': False, 'svc__kernel': 'rbf', 'svc__C': 1.9827673592309454, 'svc__gamma': 'scale'}
*** Training with the best params ***
Training done after 84.56s
Model size : 11.50MB


In [28]:
y_pred = svm.predict(X_test)
svm.evaluate(y_test, y_pred)

Evaluate for SVM classifier with TF-IDF vectorizer: 
              precision    recall  f1-score   support

         0.0       0.81      0.77      0.79       665
         1.0       0.92      0.94      0.93      1917

    accuracy                           0.90      2582
   macro avg       0.87      0.86      0.86      2582
weighted avg       0.89      0.90      0.90      2582

